In [16]:
import pymongo
import pandas as pd

data_mongo= pymongo.MongoClient('localhost:27017')

db= data_mongo['test']

In [17]:
def mongo_data(collection):
    mongoColle= db[collection]
    data= list(mongoColle.find())
    df= pd.json_normalize(data,sep='_')
    df._id=df._id.astype(str)
    return df

In [18]:
data_devi= db['devices']
device= data_devi.find({'deviceMACAddress':'0004A30B00F825B7'})
result= pd.json_normalize(device,sep='_')
result.deviceMACAddress.values[0]

'0004A30B00F825B7'

In [19]:
from bson import ObjectId
def setle_clean(select):
    de= db['settlements']
    obj= de.find_one({'_id':ObjectId(select)})
    df_setle= pd.json_normalize(obj,sep='_')
    df_setle['id_central']=df_setle.centralPoint.apply(lambda x: x[0]['_id'] if '_id' in x[0] else None)
    df_setle['latitud_c']=df_setle.centralPoint.apply(lambda x: x[0]['lat'] if 'lat' in x[0] else None)
    df_setle['longitud_c']=df_setle.centralPoint.apply(lambda x: x[0]['lng'] if 'lng' in x[0] else None)
    setle_n = df_setle[['_id','hectares','registerNumber','headsCount','name','latitud_c','longitud_c']]
    return setle_n

def selec_setle(data,select):
    return data[data._id==select]


In [22]:
d= setle_clean('63ecf27ba9f1a40025792acf')
d

,_id,hectares,registerNumber,headsCount,name,latitud_c,longitud_c
0,63ecf27ba9f1a40025792acf,1.09,None,None,MACSA,-34.164999,-64.07001


In [20]:
df_animal=mongo_data('animals')
df_devis= mongo_data('devices')
df_plot= mongo_data('plots')
df_setith= mongo_data('settlementithcounts')
df_setle= mongo_data('settlements')

In [5]:
df_devis.deviceType.unique()

array(['CARAVANA', 'COLLAR', 'PUNTO FIJO'], dtype=object)

In [21]:
df_plot['latitud']=df_plot.geoPoints.apply(lambda x: x[0]['lat'])
df_plot['longitud']=df_plot.geoPoints.apply(lambda x: x[0]['lng'])
df_plot['id_geoPoint']=df_plot.geoPoints.apply(lambda x: x[0]['_id'] if '_id' in x[0] else None)

In [22]:
df_setle['id_limits']=df_setle.limits.apply(lambda x: x[0]['_id'] if len(x)>0 else None )
df_setle['latitud']=df_setle.limits.apply(lambda x: x[0]['lat'] if len(x)>0 else None )
df_setle['longitud']=df_setle.limits.apply(lambda x: x[0]['lng'] if len(x)>0 else None )

In [23]:
df_setle['id_central']=df_setle.centralPoint.apply(lambda x: x[0]['_id'] if '_id' in x[0] else None)
df_setle['latitud_c']=df_setle.centralPoint.apply(lambda x: x[0]['lat'] if 'lat' in x[0] else None)
df_setle['longitud_c']=df_setle.centralPoint.apply(lambda x: x[0]['lng'] if 'lng' in x[0] else None)

In [15]:
df_devis[df_devis.deviceAnimalID.duplicated()]

,_id,deviceType,deviceFirmwareVersion,deviceMACAddress,deviceAnimalID,__v
35,638660af3039cf00301d200d,COLLAR,2222,0004A30B00F825B7,638660af3039cf00301d200e,0
37,638f7f2f3feed4114c00428b,COLLAR,2222,22222,62d98f17a30a66002645c2ff,0
142,63ed134ca9f1a40025795ea5,COLLAR,2.1.0,0004A30B00F8A181,63ed1054a9f1a40025795708,0
143,63ed138fa9f1a40025795eb1,COLLAR,2.1.0,0004A30B00F869B4,63ed0d7ca9f1a400257942d7,0
144,63ed13d9a9f1a40025795ec7,COLLAR,2.1.0,0004A30B00F82E81,63ed0e71a9f1a40025794898,0
145,63ed13f6a9f1a40025795edd,COLLAR,2.1.0,0004A30B00F841E4,63ed0eb2a9f1a40025794ac8,0
146,63ed1494a9f1a40025795f0b,COLLAR,2.1.0,0004A30B00F8B356,63ed0ddca9f1a400257944c0,0
162,640f3287fd164103dc37feb4,COLLAR,131313,51588585,63efe06ce429423750fcbb31,0
164,640f408f9c445d3fc452e655,CARAVANA,1111,11212123,63f66cc4d3575e0030304d2c,0


In [24]:
df_animal[df_animal._id=='638f559b47b80d0025f05f02']

,_id,animalSettlement,caravanaNumber,animalType,weight,birthDay,breed,isPregmant,dateOfBirth,observations,createdAt,updatedAt,__v,animalTags,animalSenasa
64,638f559b47b80d0025f05f02,[620e6e5e60543d0026a01f0e],PUNTO_FIJO_4,VACA,0.0,2022-12-06 14:45:47.839,DEFAULT-PUNTO-FIJO,False,NaT,NaN,2022-12-06 14:45:47.839,2022-12-06 14:45:47.839,0,[],NaN


In [ ]:
df_devis[df_devis.deviceType=='COLLAR']

In [10]:
df_devis[df_devis.deviceType=='PUNTO FIJO']

,_id,deviceType,deviceFirmwareVersion,deviceMACAddress,deviceAnimalID,__v
33,6386608e3039cf00301d1fc7,PUNTO FIJO,2222,0004A30B00F82747,638f46a8a5c1673814c1f7e2,0
34,638660a23039cf00301d1ff7,PUNTO FIJO,2222,0004A30B00F82E9C,638f557947b80d0025f05eaa,0
36,6386616d3039cf00301d20e9,PUNTO FIJO,2222,0004A30B00F88969,638f559b47b80d0025f05f02,0
147,63ed43f6a9f1a40025796d4f,PUNTO FIJO,2.1.0,0004A30B00F86440,63ed43f6a9f1a40025796d50,0
148,63ed4404a9f1a40025796d70,PUNTO FIJO,2.1.0,0004A30B00F860CC,63ed4404a9f1a40025796d71,0
149,63ed440da9f1a40025796d84,PUNTO FIJO,2.1.0,0004A30B00F89C5D,63ed440da9f1a40025796d85,0
151,63f8b232110f66003023d9d5,PUNTO FIJO,222777,PRUEBA,63f8b232110f66003023d9d6,0
153,63fc970288939900301d77bc,PUNTO FIJO,2.1.0233,MEVBE4FED3B7594,63fc970188939900301d77b7,0
155,640b79fde5142a002f656028,PUNTO FIJO,2222,0004A30B00F7FC52,640b79fde5142a002f656029,0
156,640b7a20e5142a002f656039,PUNTO FIJO,2222,0004A30B00F7FAC0,640b7a20e5142a002f65603a,0


In [32]:
df_devis[df_devis.deviceMACAddress=='4DF83DE59CD8']

,_id,deviceType,deviceFirmwareVersion,deviceMACAddress,deviceAnimalID,__v
84,63ed0e73a9f1a400257948ab,CARAVANA,2.2.0,4DF83DE59CD8,63ed0e73a9f1a400257948a6,0


In [7]:
df_setle.shape

(9, 14)

In [9]:
df_setle.iloc[0].enabledUsers

[ObjectId('6201245070900d002592d7ec'),
 ObjectId('636c17ea2bd5b2003050e02a'),
 ObjectId('6363ee75d501590030a78926')]

In [25]:
prueba= df_devis[df_devis.deviceMACAddress.isin(['0004A30B00F860CC', '0004A30B00F82E81', '0004A30B00F89C5D',
       '0004A30B00F8302C', '0004A30B00F869B4', '0004A30B00F86440',
       '0004A30B00F8A181', '0004A30B00F841E4', '0004A30B00F8B356',
       '0004A30B00F81069', '0004A30B00F842C1', '0004A30B00F8B217',
       '0004A30B00F81075', '0004A30B00F83085', '0004A30B00F842B6',
       '0004A30B00F89C52', '0004A30B00F862AF', '0004A30B00F7E0C6',
       '0004A30B00F82EB9'])]    

In [26]:
df_animal[df_animal._id.isin(prueba.deviceAnimalID)]

,_id,animalSettlement,caravanaNumber,animalType,weight,birthDay,breed,isPregmant,dateOfBirth,observations,createdAt,updatedAt,__v,animalTags,animalSenasa
100,63ed0ddca9f1a400257944c0,[63ecf27ba9f1a40025792acf],C10014,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,False,NaT,None,2023-02-15 16:52:44.079,2023-03-23 19:05:08.597,0,[],JO278-P814-8
113,63ed0e71a9f1a40025794898,[63ecf27ba9f1a40025792acf],C10013,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,False,NaT,None,2023-02-15 16:55:13.910,2023-03-27 19:55:52.914,0,[VIEJA],CC014-I034-7
120,63ed0eb2a9f1a40025794ac8,[63ecf27ba9f1a40025792acf],C10011,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,NaN,NaT,NaN,2023-02-15 16:56:18.755,2023-02-15 16:56:18.755,0,[],CC014-H762-1
162,63ed1054a9f1a40025795708,[63ecf27ba9f1a40025792acf],C10015,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,NaN,NaT,NaN,2023-02-15 17:03:16.420,2023-02-15 17:03:16.420,0,[],JO278-Q600-0
175,63ed43f6a9f1a40025796d50,[63ecf27ba9f1a40025792acf],PUNTO_FIJO_4,VACA,0.0,2023-02-15 20:43:34.927,DEFAULT-PUNTO-FIJO,False,NaT,NaN,2023-02-15 20:43:34.927,2023-02-15 20:43:34.927,0,[],NaN
176,63ed4404a9f1a40025796d71,[63ecf27ba9f1a40025792acf],PUNTO_FIJO_5,VACA,0.0,2023-02-15 20:43:48.905,DEFAULT-PUNTO-FIJO,False,NaT,NaN,2023-02-15 20:43:48.905,2023-02-15 20:43:48.905,0,[],NaN
177,63ed440da9f1a40025796d85,[63ecf27ba9f1a40025792acf],PUNTO_FIJO_6,VACA,0.0,2023-02-15 20:43:57.776,DEFAULT-PUNTO-FIJO,False,NaT,NaN,2023-02-15 20:43:57.776,2023-02-15 20:43:57.776,0,[],NaN
192,63ed0d7ca9f1a400257942d7,[63ecf27ba9f1a40025792acf],C10012,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,False,NaT,None,2023-02-15 16:51:08.808,2023-03-27 22:12:26.339,0,[LACTANCIA],CC014-H678-1
617,64235c730dac550030135fa4,[63ecf27ba9f1a40025792acf],TBOZAVI1,VACA,100.0,2022-06-18 00:00:00.000,HEREFORD,False,NaT,,2023-03-28 21:30:27.494,2023-03-28 21:30:27.494,0,[],TESTBOZAVI1
618,64235c730dac550030135fa7,[63ecf27ba9f1a40025792acf],TBOZAVI2,VACA,100.0,2022-06-18 00:00:00.000,HEREFORD,False,NaT,,2023-03-28 21:30:27.499,2023-04-03 20:06:29.531,0,[],TESTBOZAVI2


In [10]:
df_animal.animalSettlement=df_animal.animalSettlement.apply(lambda x : x[0])
df_animal.animalSettlement=df_animal.animalSettlement.astype(str)

In [11]:
masca =~df_setle._id.isin(df_animal.animalSettlement.unique())


In [12]:
df_setle[masca]#estos valores no tienen regitras con animales

,_id,enabledUsers,plots,limits,hectares,registerNumber,headsCount,name,createdAt,updatedAt,__v,centralPoint,auravantApiKey,enabled,id_limits,latitud,longitud,id_central,latitud_c,longitud_c
6,63ff75624c2d6d003084c117,[6201245070900d002592d7ec],[],"[{'_id': 63ff75624c2d6d003084c118, 'lat': -26....",61.5600,NaN,NaN,Prueba,2023-03-01 15:55:14.078,2023-03-01 15:55:45.760,0,"[{'_id': 63ff75624c2d6d003084c11c, 'lat': -26....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,False,63ff75624c2d6d003084c118,-26.853581,-65.252918,63ff75624c2d6d003084c11c,-26.858855,-65.251419
7,642b1d27cc00091984864f0a,[6201245070900d002592d7ec],[642b1d27cc00091984864f0d],[],79.9866,,0.0,prueba macsa,2023-04-03 18:38:31.434,2023-04-03 18:39:36.413,0,[{'_id': 642b1d27cc00091984864f0b}],NaN,False,None,NaN,NaN,642b1d27cc00091984864f0b,NaN,NaN
8,642c0b596490e600305e1819,[6201245070900d002592d7ec],[642c0e4a6490e600305e18eb],"[{'_id': 642c0b596490e600305e181a, 'lat': -26....",11.3300,None,NaN,Test,2023-04-04 11:34:49.097,2023-04-04 11:47:23.175,0,"[{'_id': 642c0b596490e600305e181e, 'lat': -26....",NaN,True,642c0b596490e600305e181a,-26.857838,-65.247076,642c0b596490e600305e181e,-26.859657,-65.245558


In [13]:
mascara= df_setle._id.isin(['63ff75624c2d6d003084c117','642b1d27cc00091984864f0a','642c0b596490e600305e1819'])
df_setle.drop(df_setle[mascara].index,inplace=True)

In [15]:
df_setle.to_csv('settle.csv',index=False)

In [8]:
df_plot[df_plot._id=='642c0e4a6490e600305e18eb']

,_id,auravant,virtualFenceGeoPoints,name,geoPoints,batchNumber,forage,specie,antiquity,quality,...,hectares,color,lat,lng,auravant_name,auravant_auravantPlotId,auravant_auravantSettlId,latitud,longitud,id_geoPoint
79,642c0e4a6490e600305e18eb,NaN,[],P1,"[{'_id': 642c0e4a6490e600305e18ec, 'lat': -26....",123123,LEGUMINOSAS,Alfalfa sin reposo invernal,None,EXCELENTE,...,1.08,#8db42f,NaN,NaN,NaN,NaN,NaN,-26.859093,-65.246753,642c0e4a6490e600305e18ec


In [63]:
df_setle[df_setle._id=='63ecf27ba9f1a40025792acf']

,_id,enabledUsers,plots,limits,hectares,registerNumber,headsCount,name,createdAt,updatedAt,__v,centralPoint,auravantApiKey,enabled,id_limits,latitud,longitud,id_central,latitud_c,longitud_c
5,63ecf27ba9f1a40025792acf,"[6201245070900d002592d7ec, 63ecb2e2a9f1a400257...","[63eff7fbebcff60025f5d968, 63effaa9eb90f92734f...","[{'_id': 63ecf27ba9f1a40025792ad0, 'lat': -34....",1.09,None,NaN,MACSA,2023-02-15 14:55:55.972,2023-03-23 14:29:30.396,0,"[{'_id': 63ecf27ba9f1a40025792ad4, 'lat': -34....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,True,63ecf27ba9f1a40025792ad0,-34.163851,-64.071678,63ecf27ba9f1a40025792ad4,-34.164999,-64.07001


In [82]:
#df_plot.drop(columns=['lat','lng','auravant'],inplace=True)
df_plot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   _id                       80 non-null     object 
 1   auravant                  0 non-null      float64
 2   virtualFenceGeoPoints     75 non-null     object 
 3   name                      80 non-null     object 
 4   geoPoints                 80 non-null     object 
 5   batchNumber               14 non-null     object 
 6   forage                    15 non-null     object 
 7   specie                    15 non-null     object 
 8   antiquity                 14 non-null     object 
 9   quality                   20 non-null     object 
 10  __v                       75 non-null     float64
 11  hectares                  53 non-null     float64
 12  color                     62 non-null     object 
 13  auravant_name             4 non-null      object 
 14  auravant_aur

In [81]:
df_plot.sample(8)

,_id,auravant,virtualFenceGeoPoints,name,geoPoints,batchNumber,forage,specie,antiquity,quality,__v,hectares,color,auravant_name,auravant_auravantPlotId,auravant_auravantSettlId,latitud,longitud,id_geoPoint,id_settlements
61,63eff7fcebcff60025f5dbe8,NaN,[],LA1,"[{'_id': 63eff7fbebcff60025f5d8c6, 'lng': -64....",NaN,NaN,NaN,NaN,NaN,0.0,146.14,#8db42f,NaN,NaN,NaN,-34.16558,-64.072788,63eff7fbebcff60025f5d8c6,63ecf27ba9f1a40025792acf
13,63b55ac5973f9f35c0c5b0be,NaN,[],potrero1658,"[{'_id': 63b55ac5973f9f35c0c5b0bf, 'lat': -35....",1364,LEGUMINOSAS,Alfalfa sin reposo invernal,4,EXCELENTE,0.0,NaN,NaN,NaN,NaN,NaN,-35.067196,-59.776279,63b55ac5973f9f35c0c5b0bf,None
18,63d119285237270031851b28,NaN,[],update test44,"[{'_id': 63d119285237270031851b29, 'lat': -33....",123,LEGUMINOSAS,Alfalfa con reposo intermedio,1,EXCELENTE,0.0,0.04,NaN,NaN,NaN,NaN,-33.128478,-64.349042,63d119285237270031851b29,63877cd7e1ed590030662d0a
60,63eff7fcebcff60025f5dbe2,NaN,[],LA2,"[{'_id': 63eff7fbebcff60025f5d8c1, 'lng': -64....",NaN,NaN,NaN,NaN,NaN,0.0,105.02,#8db42f,NaN,NaN,NaN,-34.172767,-64.086705,63eff7fbebcff60025f5d8c1,63ecf27ba9f1a40025792acf
24,63dcf0f00f6d500d35462947,NaN,[],lote1,"[{'_id': 63dcf0f00f6d500d3546293c, 'lng': -68....",NaN,NaN,NaN,NaN,NaN,0.0,NaN,#8db42f,NaN,NaN,NaN,-32.982545,-68.775894,63dcf0f00f6d500d3546293c,None
66,63eff7fcebcff60025f5dc23,NaN,[],CASCO LA VERDE,"[{'_id': 63eff7fbebcff60025f5d8fc, 'lng': -64....",NaN,NaN,NaN,NaN,NaN,0.0,3.91,#8db42f,NaN,NaN,NaN,-34.230406,-64.099617,63eff7fbebcff60025f5d8fc,63ecf27ba9f1a40025792acf
28,63e4f73507c91900306018e6,NaN,[],Prueba,"[{'_id': 63e4f73507c91900306018e7, 'lat': -32....",123,LEGUMINOSAS,Alfalfa con reposo intermedio,121,MUY BUENO,0.0,31.32,#dbdf00,NaN,NaN,NaN,-32.848689,-63.632171,63e4f73507c91900306018e7,620e6e5e60543d0026a01f0e
6,63aed8a1d431bd2926407806,NaN,"[[{'_id': 63aedf63ac8dad00302f6e39, 'lat': -32...",Bajo 11+Agropiro,"[{'_id': 6334700046fac000250e1f09, 'lat': -32....",None,None,None,None,EXCELENTE,0.0,10.61,NaN,NaN,NaN,NaN,-32.856105,-63.64061,6334700046fac000250e1f09,None


In [ ]:
df_plot[df_plot.color.isna()]

In [80]:
# agrego una columna para identificar a el settlements 
for i,d in df_setle.iterrows():
    id_plot=[str(x).strip() for x in d[['plots']].values[0]]
    if len(id_plot)!=0:
        df_plot.loc[df_plot._id.isin(id_plot),'id_settlements'] = df_setle.loc[i][['_id']].values[0]


In [43]:
from geopy.distance import geodesic
from geopy import Point

cordena1=tuple(x[['latitud','longitud']].values[0])
cordena2= tuple(x[['latitud_c',	'longitud_c']].values[0])

dista_km= geodesic(cordena1,cordena2).km
dista_km

0.1996762145591494

In [42]:
x[['latitud','longitud']].values[0]

array([-34.163851, -64.071678])

In [45]:
import math
def perimetro_aprox(hectarea):
    hect=hectarea
    lado = math.sqrt(hect)*10
    perim = lado*4
    return perim

In [46]:
c= perimetro_aprox(5000.00)
c

2828.42712474619